In [1]:
import os
os.chdir(os.environ['PROJECT_ROOT'])

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
pd.set_option('max.rows', 300)
pd.set_option('max.columns', 50)

In [3]:
! ls ./data/custom_panel/

batch1_variants.snps.ann.xlsx          batch2_variants_filtered_awk.indel.vcf
batch1_variants_filtered_awk.indel.vcf custom_panel.ods
batch2_variants.snps.ann.xlsx          indels.xlsx


## Custom panel: annotated fasta reads (chrX - Locus - read)

In [4]:
custom_panel = pd.ExcelFile('./data/custom_panel/custom_panel.ods',
                            engine="odf")

In [5]:
custom_panel.sheet_names

['info', 'str', 'intervals', 'fasta', 'suppl']

**Ref genome loci**

In [62]:
data = custom_panel.parse('info', header=None).rename(columns={0: 'reads'})
display(data.head())

def retrieve_reads(df):
    meta_lines = df.iloc[::2]['reads'].reset_index(drop=True)
    return pd.DataFrame({    
        'chromosome': meta_lines.apply(lambda x: x.split()[0][1:-1]),                
        'locusName': meta_lines.apply(lambda x: x.split()[1]),
        'locusStart': meta_lines.apply(lambda x: int(x.split()[2][1:-1].split('-')[0])),
        'locusEnd': meta_lines.apply(lambda x: int(x.split()[2][1:-1].split('-')[1])),
        'locusReferenceSequence': df.iloc[1::2]['reads'].reset_index(drop=True)
    })

data = retrieve_reads(data)

,reads
0,>chr1: D1S1677 (163589926-163590185)
1,CCAGTATAGTCAGGAGCTTTACTGGAAGCAACAACCCACTGTAGTG...
2,>chr2: D2S1776 (168788793-168789036)
3,ATGGTTTTTAATAAGTAGATATCAAAATGAACACAGATGTTAAGTG...
4,>chr3: D3S4529 (85803384-85803631)


**str markers: what to do with that?**

In [63]:
str_markers = custom_panel.parse('str')

In [69]:
potential_strs_index = (data.locusEnd - data.locusStart > 100)
pd.merge(
    data[potential_strs_index],
    str_markers,
    how='left',
    on=['chromosome', 'locusName', 'locusStart', 'locusEnd', 'locusReferenceSequence'],
    indicator=True
).query('_merge == "both"')

,chromosome,locusName,locusStart,locusEnd,locusReferenceSequence,repeatPattern,_merge
0,chr1,D1S1677,163589926,163590185,CCAGTATAGTCAGGAGCTTTACTGGAAGCAACAACCCACTGTAGTG...,[TTCC]15,both
1,chr2,D2S1776,168788793,168789036,ATGGTTTTTAATAAGTAGATATCAAAATGAACACAGATGTTAAGTG...,[AGAT]11,both
2,chr3,D3S4529,85803384,85803631,TATCATACATTAAAAGGGGACAAGTGCTATGCAGAAGAATGAACCA...,[AGAT]4 AAAT [AGAT]8,both
3,chr5,D5S2800,59403032,59403299,ATTACCTTCTTTATTTGATTATGTGACATTATCACCAATTTTTCTA...,[GGTA]3 [GACA]8 [GATA]3 [GATT]3,both
4,chr6,D6S474,112557851,112558118,AAAAATCAGGCTTTTATTATTCTAGAATTAAGACATGTATAGAGAC...,[TTG]3 [TTA]10,both
5,chr12,D12ATA63,107928490,107928728,CAGCACTATATCTTGATCATCAGGAGAAATGCTCTAGATCTTGGAT...,[TTG]3 [TTA]10,both
6,chr14,D14S1434,94841954,94842205,CACCATCAGTTTTTCTGAGTCTCCAGTTTGCAGGCAGCCGATTGTG...,[CTGT]3 [CTAT]10,both
7,chrX,DXS9895,7459026,7459282,GTGTTAGGCCTTTTGACTTGTCTGACAAATATTGAATGGCTCTGCT...,NaN,both
8,chrX,DXS10148,9270838,9271133,GCCTGGGAGGCAGAGATTGCAGTGAGCTGAGATCATGCGACTGCAC...,[GGAA]4 [AAGA]12 [AAAG]4 aaggaaag [AAGG]2,both
9,chrX,DXS10079,67495981,67496264,AACCTGGGTGGCAGAGGTTGCAGTGAGCTGAGATTGTGCCAATGCT...,[AGAA]17 AGAG [AGAA]3,both


In [71]:
str_markers

,chromosome,locusStart,locusEnd,locusName,repeatPattern,locusReferenceSequence
0,chr1,163589926,163590185,D1S1677,[TTCC]15,CCAGTATAGTCAGGAGCTTTACTGGAAGCAACAACCCACTGTAGTG...
1,chr2,168788793,168789036,D2S1776,[AGAT]11,ATGGTTTTTAATAAGTAGATATCAAAATGAACACAGATGTTAAGTG...
2,chr3,85803384,85803631,D3S4529,[AGAT]4 AAAT [AGAT]8,TATCATACATTAAAAGGGGACAAGTGCTATGCAGAAGAATGAACCA...
3,chr5,59403032,59403299,D5S2800,[GGTA]3 [GACA]8 [GATA]3 [GATT]3,ATTACCTTCTTTATTTGATTATGTGACATTATCACCAATTTTTCTA...
4,chr6,112557851,112558118,D6S474,[TTG]3 [TTA]10,AAAAATCAGGCTTTTATTATTCTAGAATTAAGACATGTATAGAGAC...
5,chr12,107928490,107928728,D12ATA63,[TTG]3 [TTA]10,CAGCACTATATCTTGATCATCAGGAGAAATGCTCTAGATCTTGGAT...
6,chr14,94841954,94842205,D14S1434,[CTGT]3 [CTAT]10,CACCATCAGTTTTTCTGAGTCTCCAGTTTGCAGGCAGCCGATTGTG...
7,chrX,7459026,7459282,DXS9895,NaN,GTGTTAGGCCTTTTGACTTGTCTGACAAATATTGAATGGCTCTGCT...
8,chrX,9270838,9271133,DXS10148,[GGAA]4 [AAGA]12 [AAAG]4 aaggaaag [AAGG]2,GCCTGGGAGGCAGAGATTGCAGTGAGCTGAGATCATGCGACTGCAC...
9,chrX,67495981,67496264,DXS10079,[AGAA]17 AGAG [AGAA]3,AACCTGGGTGGCAGAGGTTGCAGTGAGCTGAGATTGTGCCAATGCT...


In [82]:
from collections import defaultdict, Counter

def get_potential_strs(s, k):
    motif_pos_dict = defaultdict(list)
    motif_occ_count = Counter()
    for i in range(len(s)):
        if i+k >= len(s):
            continue
        motif = s[i:i+k]
        motif_pos_dict[motif].append(i)
        motif_occ_count.update([motif])
    #print(motif_pos_dict)
    print(motif_occ_count.most_common(5))
    
for i in range(7):
    get_potential_strs(custom_panel.parse('str').iloc[i].locusReferenceSequence, 4)

[('TTCC', 17), ('TCCT', 17), ('CCTT', 16), ('CTTC', 16), ('TATA', 5)]
[('AGAT', 15), ('GATA', 14), ('TAGA', 13), ('ATAG', 11), ('ACAC', 6)]
[('GATA', 13), ('AGAT', 12), ('ATAG', 11), ('TAGA', 11), ('AGGG', 5)]
[('ACAG', 10), ('GACA', 9), ('AGAC', 9), ('ATTT', 8), ('CAGA', 8)]
[('TAGA', 20), ('GATA', 20), ('ATAG', 19), ('AGAT', 19), ('ATGA', 6)]
[('TTAT', 10), ('TATT', 9), ('ATTA', 9), ('TGTT', 6), ('CTTG', 3)]
[('TCTA', 12), ('CTAT', 11), ('TATC', 10), ('ATCT', 9), ('TCCA', 6)]


## Indels

In [110]:
indels = pd.ExcelFile('./data/custom_panel/indels.xlsx')

In [103]:
indels.sheet_names

['indels', 'submitted', 'missed']

In [104]:
display(indels.parse('indels', header=None).shape)
display(indels.parse('submitted', header=None).head())
display(indels.parse('missed', header=None).shape)

(72, 5)

,0,1,2,3,4,5,6
0,chr1,163589926,163590185,131269,D1S1677,.,NaN
1,chr2,168788793,168789036,131309,D2S1776,.,NaN
2,chr3,85803384,85803631,131319,D3S4529,.,NaN
3,chr4,46312573,46312579,167125,GABRA2,.,NaN
4,chr4,99318159,99318165,167110,ADH1B,.,NaN


(13, 6)

In [53]:
tmp = indels.parse('indels', header=None).rename(columns={0:'c0',1:'c1',2:'c2',3:'c3',4:'c4'})
print('Max read length: ', (tmp['c2'].astype(int) - tmp['c1'].astype(int)).max())
print('Unique loci: ', tmp['c4'].nunique())

Max read length:  1
Unique loci:  28


# SNP Batches

In [16]:
from src.populations.dataset import SNPDatasetsHandler

snp_dh = SNPDatasetsHandler()

In [17]:
snp_dh.join_datasets('result', ['batch1', 'batch2'])

In [18]:
snp_dh.describe()

batch1 
46 examples, 223 variants

batch2 
44 examples, 161 variants

result 
90 examples, 138 variants



In [19]:
result = snp_dh.get('result')

In [62]:
def get_samples_distrib(level=0):
    return result.df.birth_region_id.apply(lambda x: x.split(';')[::-1][level].strip()).value_counts()
    
get_samples_distrib(3)

Кобрин                 6
Ивацевичи              5
Пинск                  5
Малорита               4
д. Туховичи            4
Минск                  3
Жабинка                3
Каменец                3
Пружаны                3
д. Будча               2
д. Рясна               2
Дрогичин               2
д. Логишин             2
Брест                  2
Ляховичи               2
д. Большие Чучевичи    1
д. Людвиново           1
д. Доброславка         1
д. Новое Попина        1
д. Дворище             1
д. Задворье            1
д. Ласица              1
д. Люсино              1
д. Хотислав            1
д. Войская             1
д. Ельня               1
Слуцк                  1
д. Воробьевка          1
д. Дмитровичи          1
Гомель                 1
д. Гвозница            1
Луганск                1
г.п. Кореличи          1
а.г. Ходосы            1
д. Ставы               1
д. Кирово              1
д. Велемче             1
д. Городец             1
Белоозерск             1
д. Патрики             1


# Fastq file

In [155]:
import glob
from collections import defaultdict

loci_occ = defaultdict(list)
for file in glob.glob('data/Данные_STR/loci_reads/*.fq'):
    s, l = file.split('.')[0].split('/')[-1].split('_')[:2]
    lines = []
    with open(file, 'r') as f:
        for line in f:
            lines.append(line.strip())
            break
    if len(lines) > 0:
        loci_occ[l].append(s)

In [158]:
list(loci_occ.keys())

['D12ATA63',
 'D14S1434',
 'D1S1677',
 'D2S1776',
 'D3S4529',
 'D5S2800',
 'D6S474',
 'DXS10075',
 'DXS10079',
 'DXS10101',
 'DXS10134',
 'DXS10146',
 'DXS10147',
 'DXS10148',
 'DXS7133',
 'DXS7424',
 'DXS8377',
 'DXS9895',
 'GATA172D05',
 'DXS7423',
 'D13S317',
 'VWA',
 'D12S391',
 'D4S2408',
 'D5S2500',
 'DXS10135']

In [148]:
loci_occ

defaultdict(list,
            {'ADH1B': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'ADH1C': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'ALDH2': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'ANKRD11': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'ASIP': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'BNC2': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'CHRNA3': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'COMT': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'CYP2E15B': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'D12ATA63': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'D14S1434': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'D1S1677': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'D2S1776': ['S14', 'S20', 'S26', 'S32', 'S38', 'S44', 'S8'],
             'D3S4529': ['S14', '

In [111]:
indels_df = indels.parse('submitted', header=None).rename(
    columns={0:'0',1:'1',2:'2',3:'3',4:'4'}
)
indels_df.head()

,0,1,2,3,4,5,6
0,chr1,163589926,163590185,131269,D1S1677,.,NaN
1,chr2,168788793,168789036,131309,D2S1776,.,NaN
2,chr3,85803384,85803631,131319,D3S4529,.,NaN
3,chr4,46312573,46312579,167125,GABRA2,.,NaN
4,chr4,99318159,99318165,167110,ADH1B,.,NaN


In [114]:
indels_df['id'] = indels_df['0'] + ':' + indels_df['1'].astype('str') + '-' + indels_df['2'].astype('str') + ' ' + indels_df['4']
indels_df.head()

,0,1,2,3,4,5,6,id
0,chr1,163589926,163590185,131269,D1S1677,.,NaN,chr1:163589926-163590185 D1S1677
1,chr2,168788793,168789036,131309,D2S1776,.,NaN,chr2:168788793-168789036 D2S1776
2,chr3,85803384,85803631,131319,D3S4529,.,NaN,chr3:85803384-85803631 D3S4529
3,chr4,46312573,46312579,167125,GABRA2,.,NaN,chr4:46312573-46312579 GABRA2
4,chr4,99318159,99318165,167110,ADH1B,.,NaN,chr4:99318159-99318165 ADH1B
